# HOW TO USE


In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/NLP/experiments/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/NLP/experiments


In [2]:
# 깃허브의 install_packages.sh install 하기
!chmod +x install_packages.sh
!./install_packages.sh

In [3]:
file_path = "/content/drive/MyDrive/Colab Notebooks/NLP/experiments/"
file_name = "ewha.pdf"

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/NLP/experiments/module
# 모듈 있는곳
from google.colab import files
# uploaded = files.upload()  # .env 파일을 업로드. 로컬에 저장되어있으면 안해도 됨
from dotenv import load_dotenv
import os

with open('.env', 'w') as f: #.env 파일에 API KEY 설정해야 돌아갑니다!! 이 코드는 파일에 직접 apikey 저장
    f.write('UPSTAGE_API_KEY=_____YOUR KEY HERE_____\n') 

/content/drive/MyDrive/Colab Notebooks/NLP/experiments/module


In [ ]:
load_dotenv()
os.getenv("UPSTAGE_API_KEY") # 키 확인

---
# 실험

In [6]:
%cd /content/drive/MyDrive/Colab Notebooks/NLP/experiments/module
#모듈 코드 있는곳으로 이동해야 합니다.
from docs_loaders import DocumentLoader
from text_splitters import TextSplitter
from vector_stores import VectorStore
from retrievers import Retriever
from langchain.schema.runnable import RunnablePassthrough
# from langchain_community.vectorstores import FAISS
# from langchain_community.document_loaders import TextLoader
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_upstage import (
    # UpstageLayoutAnalysisLoader,
    UpstageEmbeddings,
    ChatUpstage
    )
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

/content/drive/MyDrive/Colab Notebooks/NLP/experiments/module


In [10]:
# 실험 셋업
%cd /content/drive/MyDrive/Colab Notebooks/NLP/experiments/module

file_path = "/content/drive/MyDrive/Colab Notebooks/NLP/experiments/"
file_name = "ewha.pdf" # 파일 이름

document_loader=DocumentLoader(file_path+file_name) # 도큐먼트 로딩 인스턴스 생성
docs = document_loader.pdfplumber_loader() # PDF PLUMBER사용해서 로딩

text_splitter = TextSplitter() # Splitter 인스턴스 생성
text_splitter.setup_document(docs) # 로딩된 docs를 올리기
splits = text_splitter.split_into_sections() # Split 함


#--- FAISS Local저장 안한 경우 ---
vec_store = VectorStore() # 벡터스토어 인스턴스 생성
vec_store.setup_FAISS(splits) # FAISS에 splits를 임베딩해서 저장함
db = vec_store.save_db_local("/content/drive/MyDrive/Colab Notebooks/NLP/experiments/faiss_db/test") # 로컬에 벡터스토어 저장. 최초한번만!

#--- 임베딩 매번 새로 하지 말고 로드해서 쓰세요! --- 돈 많이 나가요
vec_store = VectorStore() # 벡터스토어 인스턴스 생성
db = vec_store.load_db_local("/content/drive/MyDrive/Colab Notebooks/NLP/experiments/faiss_db/chapter_splitter") # 기존 split되어있는것 불러옴


retriever_instance= Retriever(vec_store.db, vec_store.embeddings) #retriever 인스턴스 생성
retriever = retriever_instance.vectorstore_retriever() #RETRIEVER 생성

/content/drive/MyDrive/Colab Notebooks/NLP/experiments/module
{0: '60944d5d-eb93-4ab7-a279-19927c33af99', 1: '3421fcbd-aac5-44d6-a708-cd851643c509', 2: 'c281f2fd-2bbc-4ff8-9de9-22b26f53f601', 3: 'cf0d5b26-0e69-4f29-aa54-e6a6f44b47f2', 4: '2561adb7-062e-4721-9ed5-f301d6742cad', 5: 'd8772dca-3c9d-42d2-805c-06401c90b8fd', 6: '7ca80864-2ad6-4e45-a462-d69b0899a97e', 7: '772c4491-c841-4d27-add4-5410c5b4b2ca', 8: 'ffb16cc3-de0b-4211-a81d-9b8245adfda3', 9: '5620472c-8440-47ab-ab29-e43a01c1f3a3', 10: '82a91741-0470-42bc-b96f-5923d1fe77b6', 11: 'f1002169-55ee-42a6-9ee1-b2a6fa968703', 12: '7cbd3aab-39d2-4a6c-b16b-670308698b20', 13: 'eab5f912-e089-466b-8c20-febae7d8cd9a', 14: 'b4323971-46ef-4f41-9d25-0a5fc6af65d8', 15: '40c620d8-f003-4ba4-938b-93a2e2964121', 16: '07bcdbb8-387f-48b7-baed-d4ab9ddb4ae4', 17: '9ac96c8d-39c2-4d1a-a49f-a404d5e6a1e8', 18: '72248c4c-5a38-4f10-ae9e-ea6d4f507b95', 19: '830e3938-eb1d-4a4d-aa36-a42c6b09247a', 20: 'a2c03ec8-99ad-4fd2-b319-a9e4dd8e7f5a', 21: 'ff883d9f-b666-4738

In [11]:
prompt_template = PromptTemplate.from_template(
    """
    You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question considering the fact of the question.
    If you don't know the answer, just say that you don't know, don't try to make up an answer.

    YOU SHOULD FOLLOW THE FORMAT BELOW:
    (Answer Letter) [Your Answer]

    For example:
    (A) Example Answer

    ---
    Question: {question}
    ---
    Context: {context}
    """
)

llm = ChatUpstage(api_key = os.getenv("UPSTAGE_API_KEY"))

chain =({"question": RunnablePassthrough(), "context": retriever}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [12]:
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/NLP/experiments/Team4_Testsamples.csv")

In [13]:
data

,prompts,answers
0,QUESTION1) 건축학전공의 수업연한은?\n(A) 4년\n(B) 5년\n(C) ...,(B)
1,QUESTION2) 학사경고를 받는 학점은?\n(A) 1.6\n(B) 1.8\n(C...,(A)
2,QUESTION3) 계절학기 취득 가능 학점은?\n(A) 2학점\n(B) 4학점\n...,(C)
3,QUESTION4) 중앙행정기관이 아닌 기관은?\n(A) 교무처\n(B) 인재개발처...,(B)
4,QUESTION5) 체육특기자의 대회출전 및 훈련으로 인한 출석대체인정은 1학기 수...,(D)
5,QUESTION6) 교육과목의 종류와 학점을 정하는 사람은?\n(A) 총장\n(B)...,(A)
6,QUESTION7) 전과 허가가 가능한 기간은?\n(A) 매 학년도초\n(B) 매 ...,(B)
7,QUESTION8) 이화여자대학교의 교육 이념은 무엇을 기반으로 하고 있습니까?\n...,(B)
8,QUESTION9) 이화여자대학교 학칙에 따르면 출산 및 육아로 인한 휴학 허용 기...,(B)
9,QUESTION10) 이화여자대학교의 졸업 요건 중 영어 관련 기준은 다음 중 무엇...,(C)


In [14]:
data["Response"] = data["prompts"].apply(lambda prompt: chain.invoke(prompt)) #응답 받기

In [ ]:
data

,prompts,answers,Response
0,QUESTION1) 건축학전공의 수업연한은?\n(A) 4년\n(B) 5년\n(C) ...,(B),(B) 5년
1,QUESTION2) 학사경고를 받는 학점은?\n(A) 1.6\n(B) 1.8\n(C...,(A),(B) 1.8
2,QUESTION3) 계절학기 취득 가능 학점은?\n(A) 2학점\n(B) 4학점\n...,(C),(C) 6학점
3,QUESTION4) 중앙행정기관이 아닌 기관은?\n(A) 교무처\n(B) 인재개발처...,(B),(A) 교무처
4,QUESTION5) 체육특기자의 대회출전 및 훈련으로 인한 출석대체인정은 1학기 수...,(D),(D) 2분의 1
5,QUESTION6) 교육과목의 종류와 학점을 정하는 사람은?\n(A) 총장\n(B)...,(A),(A) 총장
6,QUESTION7) 전과 허가가 가능한 기간은?\n(A) 매 학년도초\n(B) 매 ...,(B),(B) 매 학년도말
7,QUESTION8) 이화여자대학교의 교육 이념은 무엇을 기반으로 하고 있습니까?\n...,(B),(B) 기독교 정신과 학문 연구
8,QUESTION9) 이화여자대학교 학칙에 따르면 출산 및 육아로 인한 휴학 허용 기...,(B),(D) 4년
9,QUESTION10) 이화여자대학교의 졸업 요건 중 영어 관련 기준은 다음 중 무엇...,(C),(B) 영어 인증시험 점수를 제출


In [17]:
%cd /content/drive/MyDrive/Colab Notebooks/NLP/experiments/module
#모듈 코드 있는곳으로 이동해야 합니다.
import utils

/content/drive/MyDrive/Colab Notebooks/NLP/experiments/module


In [18]:
data["pred_answer"] = data["Response"].apply(utils.extract_answer)
acc=utils.calculate_accuracy(data)
data

,prompts,answers,Response,pred_answer,Correct
0,QUESTION1) 건축학전공의 수업연한은?\n(A) 4년\n(B) 5년\n(C) ...,(B),(B) 5년,B,True
1,QUESTION2) 학사경고를 받는 학점은?\n(A) 1.6\n(B) 1.8\n(C...,(A),(A) 1.6,A,True
2,QUESTION3) 계절학기 취득 가능 학점은?\n(A) 2학점\n(B) 4학점\n...,(C),(C) 6학점,C,True
3,QUESTION4) 중앙행정기관이 아닌 기관은?\n(A) 교무처\n(B) 인재개발처...,(B),(B) 인재개발처,B,True
4,QUESTION5) 체육특기자의 대회출전 및 훈련으로 인한 출석대체인정은 1학기 수...,(D),(D) 2분의 1,D,True
5,QUESTION6) 교육과목의 종류와 학점을 정하는 사람은?\n(A) 총장\n(B)...,(A),(A) 총장,A,True
6,QUESTION7) 전과 허가가 가능한 기간은?\n(A) 매 학년도초\n(B) 매 ...,(B),(B) [매 학년도말],B,True
7,QUESTION8) 이화여자대학교의 교육 이념은 무엇을 기반으로 하고 있습니까?\n...,(B),(B) 기독교 정신과 학문 연구,B,True
8,QUESTION9) 이화여자대학교 학칙에 따르면 출산 및 육아로 인한 휴학 허용 기...,(B),(B) 2년,B,True
9,QUESTION10) 이화여자대학교의 졸업 요건 중 영어 관련 기준은 다음 중 무엇...,(C),(B) 영어 인증시험 점수를 제출합니다.,B,False


In [19]:
acc

85.0